DRIVE SETUP

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


DATA MINING CODE

In [0]:
import pandas as pd
import torch
from multiprocessing import Process, Queue, Array, Value
import time
import pandas_datareader as pdr
# 종목 타입에 따라 download url이 다름. 종목코드 뒤에 .KS .KQ등이 입력되어야해서 Download Link 구분 필요
stock_type = {
'kospi': 'stockMkt',
'kosdaq': 'kosdaqMkt'
}
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    code = df.query("name=='{}'".format(name))['code'].to_string(index=False)
# 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
    code = code.strip()
    return code
# download url 조합
def get_download_stock(market_type=None):
    market_type = stock_type[market_type]
    download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
    download_link = download_link + '?method=download'
    download_link = download_link + '&marketType=' + market_type
    df = pd.read_html(download_link, header=0)[0]
    return df;
# kospi 종목코드 목록 다운로드
def get_download_kospi():
    df = get_download_stock('kospi')
    df.종목코드 = df.종목코드.map('{:06d}.KS'.format)
    return df
# kosdaq 종목코드 목록 다운로드
def get_download_kosdaq():
    df = get_download_stock('kosdaq')
    df.종목코드 = df.종목코드.map('{:06d}.KQ'.format)
    return df
# kospi, kosdaq 종목코드 각각 다운로드
kospi_df = get_download_kospi()
kosdaq_df = get_download_kosdaq()
# data frame merge
code_df = pd.concat([kospi_df, kosdaq_df])
# data frame정리
code_df = code_df[['회사명', '종목코드']]
# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
size=len(code_df)
iter=0
err=0

def get_url(item_name, code_df):
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    code=code[1:]
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    print("요청 URL = {}".format(url))
    return url # 신라젠의 일자데이터 url 가져오기

def get_url_by_code(itemcode,code_df):
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=itemcode)
    print("요청 URL = {}".format(url))
    return url

def find_by_code_A(itemcode):
    url = get_url_by_code(itemcode, code_df) # 일자 데이터를 담을 df라는 DataFrame 정의
    df = pd.DataFrame() # 1페이지에서 20페이지의 데이터만 가져오기
    for page in range(1, 1000):
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
        df = df.dropna()
    df.drop_duplicates(['날짜'], keep='first', inplace=True)
    return df


threads=Array('i',range(6))
thrds=[None,None,None,None,None,None]

def threaded_search(com_code, num):
    torch.multiprocessing.freeze_support()
    threads[num]=1
    print('Search for a code:{0} started.'.format(com_code))
    df = find_by_code_A(com_code)
    df.to_csv('/content/gdrive/Shared drives/STOCK/DATA_5.30/' + str(com_code) + '.csv', mode='w', encoding='euc-kr')
    print('Search for a code:{0} finished.'.format(com_code))
    threads[num]=0

def chk_all_threads_started(thrds,threads):
    for i in range(len(threads)):
        if(threads[i]==0):
            return False
    return True;

def chk_all_threads_dead(thrds,threads):
    for i in range(len(thrds)):
        if thrds[i].is_alive():
            return False
    return True

def join_threads(thrds):
    for i in thrds:
        i.join()

if __name__ == '__main__':
    for i in range(6):
        threads[i]=0
    for i,row in code_df.iterrows():
        iter+=1
        val=row['code'][:-3]
        print(val)
        got_thread=0

        while got_thread==0 :
            for j in range(len(threads)):
                if threads[j] == 0:
                    thrds[j] = Process(target = threaded_search,args=(val, j))
                    thrds[j].start()
                    got_thread=1
                    print("Process : " + str(thrds[j]))
                    threads[j]=1
                    break
        if chk_all_threads_started(thrds,threads):
            join_threads(thrds)
            time.sleep(1)
            while not chk_all_threads_dead(thrds,threads):
                pass
            for j in range(len(threads)):
                threads[j]=0

        print(str(iter)+'/'+str(size), "err = "+str(err))


    print('err='+str(err))


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


005560
Process : <Process(Process-1, started)>
1/3619 err = 0
101380
Search for a code:005560 started.
Process : <Process(Process-2, started)>
2/3619 err = 0
114140
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=005560
Search for a code:101380 started.
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=101380
Process : <Process(Process-3, started)>
3/3619 err = 0
064900
Search for a code:114140 started.
Process : <Process(Process-4, started)>
4/3619 err = 0
028780
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=114140
Search for a code:064900 started.
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=064900
Process : <Process(Process-5, started)>
5/3619 err = 0
085450
Search for a code:028780 started.
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=028780
Search for a code:085450 started.
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=085450
Process : <Process(Process-6, started)>
Search for a code:028780 finished.
Search for a code:11